<a href="https://colab.research.google.com/github/mariacosioleon/IA20182/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

In [4]:
from google.colab import files
#se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
files.upload()

#path_to_fileDL ='/content/Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt'

path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')

99993/99993 [==============================] - 0s 1us/step


In [5]:
text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        99993 carácteres
El texto está compuesto de estos 62 carácteres:
['\n', ' ', '!', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres:

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  'K' :  20,
  'L' :  21,
  'M' :  22,
  'N' :  23,
  'O' :  24,
  'P' :  25,
  'Q' :  26,
  'R' :  27,
  'S' :  28,
  'T' :  29,
  'U' :  30,
  'V' :  31,
  'W' :  32,
  'X' :  33,
  'Y' :  34,
  'Z' :  35,
  'a' :  36,
  'b' :  37,
  'c' :  38,
  'd' :  39,
  'e' :  40,
  'f' :  41,
  'g' :  42,
  'h' :  43,
  'i' :  44,
  'j' :  45,
  'k' :  46,
  'l' :  47,
  'm' :  48,
  'n' :  49,
  'o' :  50,
  'p' :  51,
  'q' :  52,
  'r' :  53,
  's' :  54,
  't' :  55,
  'u' :  56,
  'v' :  57,
  'w' :  58,
  'x' :  59,
  'y' :  60,
  'z' :  61,


Ahora tenemos una representación de entero (integer) para cada carácter que podemos ver ejecutando.

In [7]:
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: "That, poor contempt, or claim'd thou slept so fait"
array([29, 43, 36, 55,  4,  1, 51, 50, 50, 53,  1, 38, 50, 49, 55, 40, 48,
       51, 55,  4,  1, 50, 53,  1, 38, 47, 36, 44, 48,  3, 39,  1, 55, 43,
       50, 56,  1, 54, 47, 40, 51, 55,  1, 54, 50,  1, 41, 36, 44, 55])


## **Preparación de los datos para ser usados por la RNN**

Para entrenar el modelo prepararemos unas secuencias de caracteres como entradas y salida de un tamaño determinado. En nuestro ejemplo hemos definido el tamaño de 100 caracteres con la variable seq_length (que bien puede  modificarse).

Empezamos dividiendo el texto que tenemos en secuencias deseq_length+1de caracteres con las cuales luego contruiremos los datos de entrenamiento compuestos por las entradas de seq_lengthcaracteres y las salidas correspondientes que contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha. Volviendo al ejemplo de “Hola” anterior, y suponiendo un seq_length=3, la secuencia de entrada será “Hol”, y la de salida será “ola”.

Usaremos la función tf.data.Dataset.from_tensor_slices, que crea un conjunto de datos con el contenido del tensortext_as_int que contiene el texto, al que podremos aplicar el método batch( ) para dividir este conjunto de datos en secuencias de  seq_length+1  de índice de caracteres:



In [9]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Podemos comprobar que sequences contiene el texto dividido en paquetes de 101 caracteres como esperamos (por ejemplo mostremos las 10 primeras secuencias):

In [10]:
for item in sequences.take(10):
  print(repr(' '.join(idx2char[item.numpy()])))

"T h a t ,   p o o r   c o n t e m p t ,   o r   c l a i m ' d   t h o u   s l e p t   s o   f a i t h f u l , \n I   m a y   c o n t r i v e   o u r   f a t h e r ;   a n d ,   i n   t h e i r   d e f e"
'a t e d   q u e e n , \n H e r   f l e s h   b r o k e   m e   a n d   p u t t a n c e   o f   e x p e d i t i o n   h o u s e , \n A n d   i n   t h a t   s a m e   t h a t   e v e r   I   l a m e n t  '
't h i s   s t o m a c h , \n A n d   h e ,   n o r   B u t l y   a n d   m y   f u r y ,   k n o w i n g   e v e r y t h i n g \n G r e w   d a i l y   e v e r ,   h i s   g r e a t   s t r e n g t h   a'
"n d   t h o u g h t \n T h e   b r i g h t   b u d s   o f   m i n e   o w n . \n \n B I O N D E L L O : \n M a r r y ,   t h a t   i t   m a y   n o t   p r a y   t h e i r   p a t i e n c e . ' \n \n K I N"
'G   L E A R : \n T h e   i n s t a n t   c o m m o n   m a i d ,   a s   w e   m a y   l e s s   b e \n a   b r a v e   g e n t l e m a n   a n d   j o i n e r :   h e   t 

De esta secuencia se obtiene el conjunto de datos de training que contenga tanto los datos de entrada (desde la posición 0 a la 99) como los datos de salida (desde la posición 1 a la 100). Para ello se crea una función que realiza esta tarea y se aplica a todas las secuencias usando el método map( )de la siguiente forma:

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

En este punto, dataset contiene un conjunto de parejas de secuencias de texto (con la representación numérica de los caracteres), donde el primer componente de la pareja contiene un paquete con una secuencia de 100 caracteres del texto original y la segunda su correspondiente salida, también de 100 caracteres. Podemos comprobarlo visualizándolo por pantalla (por ejemplo mostrando la primera pareja):

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "That, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their def"
Target data: "hat, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their defe"


En este punto del código disponemos de los datos de entrenamiento en el tensor dataset en forma de parejas de secuencias de 100 integersde 64 bits que representan un carácter del vocabulario:

In [13]:
print (dataset)

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


En realidad los datos ya están preprocesados en el formato que se requiere para ser usados en el entreno de la red neuronal, pero recordemos que en redes neuronales los datos se agrupan en batches antes de pasarlos al modelo. En nuestro caso hemos decidido un tamaño de batch de 64, que nos facilita la explicación, este es un hiperparámetro importante de ajustar correctamente teniendo en cuenta diferentes factores, como el tamaño de la memoria disponible, por poner un ejemplo. En este código, para crear los batches de parejas de secuencias hemos considerado usar tf.dataque además nos permite barajar las secuencias previamente:

In [14]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


Recapitulando, ahora en el tensor dataset disponemos de los datos de entrenamiento ya listos para ser usados para entrenar el modelo: batches compuestos de 64 parejas de secuencias de 100 integers de 64 bits que representan el carácter correspondiente en el vocabulario.

## Construcción del modelo RNN
Para construir el modelo usaremos tf.keras.Sequential que ya conocemos. Usaremos una versión mínima de RNN para facilitar la explicación, que contenga solo una capa LSTM. En concreto definimos una red de solo 3 capas:

In [15]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

La primera capa es de tipo Word Embedding que mapea cada carácter de entrada en un vector Embedding. Esta capa tf.keras.layers.Embedding permite especificar varios argumentos que se pueden consultar en todo detalle en el manual de TensorFlow[21].

En nuestro caso el primero que especificamos es el tamaño del vocabulario, indicado con el argumento vocab_size, que indica cuantos vectores Embeddingtendrá la capa.  A continuación indicamos las dimensiones de estos vectores Embeddingmediante el argumento embedding_dim, que en nuestro caso hemos decidido que sea 256.  Finalmente se indica el tamaño del batchque usaremos para entrenar, en nuestro caso 64.

La segunda capa es de tipo LSTM introducida anteriormente en este capítulo. Esta capa tf.keras.layers.LSTM tiene varios argumentos posibles que se pueden consultar en el manual de TensorFlow[22], aquí solo usaremos algunos y dejamos los valores por defecto del resto. Quizás el más importante es el número de neuronas recurrentes que se indica con el argumento units y que en nuestro caso hemos decidido que sea 1024 neuronas.

Con return_sequencese indica que queremos predecir el carácter siguiente a todos los caracteres de entrada, no solo el siguiente al último carácter.

El argumento statefulindica, explicado de manera simple, el uso de las capacidades de memoria de la red entre batches. Si este argumento está instanciado a falsese indica que a cada nuevo batchse inicializan las memory cellscomentadas anteriormente, mientras que si está a truese está indicando para cada batchse mantendrán las actualizaciones hechas durante la ejecución del bachanterior.

El último argumento que usamos es recurrent_kernel, donde indicamos cómo se deben inicializar los pesos de las matrices internas de la red. En este caso usamos la distribución uniforme glorot_uniform, habitual en estos casos.

Finalmente la última capa es de tipo Dense, ya explicada previamente en este libro. Aquí es importante el argumento units que nos dice cuantas neuronas tendrá la capa y que nos marcará la dimensión de la salida. En nuestro caso será igual al tamaño de nuestro vocabulario (vocab_size).

Como siempre, es interesante usar el método summary()para visualizar la estructura del modelo:

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           15872     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 62)            63550     
                                                                 
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


Podemos comprobar que la capa LSTM consta de muchos parámetros (más de 5 millones) como era de esperar. Intentemos analizar un poco más esta red neuronal. Para cada carácter de entrada (transformado a su equivalente numérico), el modelo busca su vector de Embedding correspondiente y luego ejecuta la capa LSTM con este vector Embeddingcomo entrada. A la salida de la LSTM aplica la capa Densepara decidir cual es el siguiente carácter.

Inspeccionemos las dimensiones de los tensores para poder comprender más a fondo el modelo. Fijemonos en el primer batchdel conjunto de datos de entrenamiento y observemos su forma:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


Podemos comprobar que la capa LSTM consta de muchos parámetros (más de 5 millones) como era de esperar. Intentemos analizar un poco más esta red neuronal. Para cada carácter de entrada (transformado a su equivalente numérico), el modelo busca su vector de Embedding correspondiente y luego ejecuta la capa LSTM con este vector Embeddingcomo entrada. A la salida de la LSTM aplica la capa Dense para decidir cual es el siguiente carácter.

Inspeccionemos las dimensiones de los tensores para poder comprender más a fondo el modelo. Fijemonos en el primer batch del conjunto de datos de entrenamiento y observemos su forma:

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 62) # (batch_size, sequence_length, vocab_size)


Revisemos ahora la capa densa de esta red neuronal la cual no tiene una función de activación softmax como la capa densa que tienen algunas redes neuronales. De aquí que retorne el vector con un indicador de “evidencia” para cada carácter.

El siguiente paso es elegir uno de los caracteres. Sin entrar en detalle, no se eligirá el carácter más “probable” (mediante argmax) puesto que el modelo puede entrar en un bucle. Lo que se hará es obtener una muestra de la distribución de salida. Pruébelo para el primer ejemplo en el batch:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

Con tf.random.categoricalse obtiene una muestra de una distribución categórica y con squeezese elimina la dimensiones del tensor de tamaño 1. De esta manera en cada instante de tiempo se obtiene una predicción del índice del siguiente carácter.

In [22]:
sampled_indices_characters

array([48, 50, 50, 41, 55,  3, 34,  9, 24, 12, 37,  3, 29, 30, 37, 11, 57,
       41,  4, 47, 46, 39, 33,  8, 60, 29, 51, 58, 50, 61, 29, 36, 23, 47,
       61, 14, 55, 47, 15,  2, 54, 28, 19, 25, 33, 17,  1, 21, 32, 35, 28,
       13, 24, 55, 61, 42, 23,  9, 10, 16, 61, 30, 43, 45,  2, 37, 40,  6,
       13, 52, 58, 60, 54, 20, 27, 37, 23, 26,  5, 12, 18,  2, 57, 53,  9,
       56, 32, 30, 48, 16,  3, 52, 49, 52, 15, 58, 57, 15, 32, 32])

## Entrenamiento del modelo RNN
En este punto, el problema puede tratarse como un problema de clasificación estándar para el que debemos definir la función de Lossy el optimizador.

Para la función de Loss usaremos la función estándar tf.keras.losses.sparse_categorical_crossentropy dado que estamos considerando datos categóricos. Dado que el retorno hemos visto que se trata de unos valores de verisimilitud (no de probabilidades como si hubiéramos ya aplicado softmax) se instanciará el argumentofrom_logits a True.

Entrenar el modelo

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

En cuanto al optimizador usaremos tf.keras.optimizers.Adam con los argumentos por defecto del optimizador Adam.

Con esta función de loss definida y usando el optimizador Adam con sus argumentos por defecto, ya podemos llamar al método compile () de la siguiente manera:

In [24]:
model.compile(optimizer='adam', loss=loss)

En este ejemplo aprovecharemos para usar los Checkpoints[23], una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.  En nuestro caso, cuando entrenamos modelos Deep Learning, el Checkpoint lo forman básicamente los pesos del modelo. Estos Checkpoint se pueden usar también para hacer predicciones tal cual como haremos en este ejemplo.

La librería de Keras proporciona Checkpoints a través de la API Callbacks.   Concretamente usaremos tf.keras.callbacks.ModelCheckpoint[24]para especificar cómo salvar los Checkpointsa cada epoch durante el entrenamiento, a través de un argumento en el método fit() del modelo.

En el código debemos especificar el directorio en el que se guardarán los Checkpoints que salvaremos y el nombre del fichero (que le añadiremos el número de epoch para nuestra comodidad):

Configurar Checkpoints

In [26]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
15/15 [==============================] - 80s 5s/step - loss: 3.4572
Epoch 2/50
15/15 [==============================] - 74s 5s/step - loss: 3.2118
Epoch 3/50
15/15 [==============================] - 73s 5s/step - loss: 3.0460
Epoch 4/50
15/15 [==============================] - 72s 5s/step - loss: 2.6836
Epoch 5/50
15/15 [==============================] - 75s 5s/step - loss: 2.4725
Epoch 6/50
15/15 [==============================] - 72s 5s/step - loss: 2.3676
Epoch 7/50
15/15 [==============================] - 73s 5s/step - loss: 2.2942
Epoch 8/50
15/15 [==============================] - 72s 5s/step - loss: 2.2267
Epoch 9/50
15/15 [==============================] - 72s 5s/step - loss: 2.1650
Epoch 10/50
15/15 [==============================] - 73s 5s/step - loss: 2.1092
Epoch 11/50
15/15 [==============================] - 73s 5s/step - loss: 2.0544
Epoch 12/50
15/15 [==============================] - 72s 5s/step - loss: 2.0046
Epoch 13/50
15/15 [==============================

### Generación de texto

Generación de texto usando el modelo RNN
Ahora que tenemos ya entrenado el modelo pasemos a usarlo para generar texto. Para mantener este paso de predicción simple, vamos a usar un tamaño de batch de 1. Debido a la forma en que se pasa el estado de la RNN de un instante de tiempo al siguiente, el modelo solo acepta un tamaño de batch fijo una vez construido. Por ello, para poder ejecutar el modelo con un tamaño de batch diferente, necesitamos reconstruir manualmente el modelo con el método build( )del modelo y restaurar sus pesos desde el Checkpoints(cogemos el ultimo con tf.train.latest_checkpoint ()):

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Ahora que tenemos el modelo entrenado y preparado para usar, generaremos texto a partir de una palabra de partida con el siguiente código:

In [ ]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

El código empieza con inicializaciones como: definir el número de caracteres a predecir con la variable num_generate,convertir la palabra inicial (start_string) a su correspondiente representación numérica y preparan lo tensores necesarios:

Usando la misma idea del código original char-rnn[25]de Andrey Karpathy, se usa una variable temperature para decidir cómo de conservador en sus predicciones queremos que se comporte nuestro modelo. En nuestro ejemplo la hemos inicializado a 0.5:

Con “temperaturas altas” (hasta 1) se permitirá más creatividad al modelo para generar texto pero a costa de más errores (por ejemplo, errores ortográficos, etc.). Mientras que con “temperaturas bajas” habrá menos errores pero el modelo mostrará poca creatividad. Propongo que el lector pruebe con diferentes valores y vea su efecto. Incluso le propongo que use el datasetShakespeare.txt[26](de tamaño mucho mayor que el presentado en este libro) que Andrey Karpathy usa en su ejemplo original.

A partir de este momento empieza el bucle para generar los caracteres que le hemos indicado (que usa el carácter de entrada la primera vez) y luego sus própias predicciones como entrada a cada iteración al modelo RNN:

Recordemos que estamos en un batchde 1 pero el modelo retorna el tensor del batchcon las dimensiones que lo habíamos entrenado y por tanto debemos reducir la dimensión batch:



Luego, se usa una distribución categórica para calcular el índice del carácter predicho:



Este carácter acabado de predecir se usa como nuestra próxima entrada al modelo, retroalimentando el modelo para que ahora tenga más contexto (en lugar de una sola letra). Después de predecir la siguiente letra, se retroalimenta nuevamente, y así sucesivamente de manera que es cómo aprende a medida que se obtiene más contexto de los carácteres predichos previamente:



Ahora que se ha descrito cómo se ha programado la función generate_txtprobemos cómo se comporta el modelo.

Empecemos con una palabra que no conoce el corpus, por ejemplo “Alcohol”, que nada tiene que ver con Deep Learning:



Como vemos el modelo no es capaz de generar ningún texto que tenga ningún parecido a un posible texto relacionado con el tema.

Probemos ahora con una palabra como “modelo” o “activación” a ver que pasa:

In [ ]:
print(generate_text(model, start_string=u"a"))

En resumen, el modelo presentado parece que ha aprendido a generar texto de manera interesante, teniendo en cuenta el reducido dataset inicial con el que se ha entrenado. Como ya hemos avanzado, proponemos que el lector pruebe con otros conjuntos de datos de tipo texto. Por ejemplo en el artículo “The Unreasonable Effectiveness of Recurrent Neural Network” del blog de Andrey Karpathy[27]el lector puede encontrar varios ejemplos de datos de tipo texto que el lector puede usar directamente simplemente cambiando la URL del fichero de texto de entrada al código propuesto en este capítulo.

Hasta aquí un ejemplo muy simple pero que espero que haya sido útil al lector para comprender la idea que hay detrás de las redes neuronales recurrentes. Un tipo de arquitectura que solo hace pocos años que han iniciado su andadura con resultados impresionante en un amplio abanico de tareas como machine translation[28](2015), language modeling[29](2015) o speech recognition[30](2013) por poner algunos ejemplos.

Es sin duda una de las áreas de investigación más activas en Deep Learning en estos momentos, en la que incluso nuestro grupo de investigación está realizando aportaciones[31].  Pero también es un área que genera mucho debate, al poderse crear sistemas que escriben prosa de manera convincente como el presentado recientemente por OpenAI[32]. Un modelo entrenado con miles de millones de palabras para poder crear artículos “creibles”, que muestra cómo estos algoritmos podrían usarse para engañar a las personas a gran escala, automatizando por ejemplo la generación de noticias falsas en redes sociales.